In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import geodatasets
import matplotlib.pyplot as plt

## Visualize the datasets 

Plot the dust deposition rate of the empirical dataset in the measurement locations and the simulated data on the global grid.

In [ ]:
df_empirical_Holocene = pd.read_csv("df_empirical_Holocene.csv")
df_empirical_LGM = pd.read_csv("df_empirical_LGM.csv")

In [ ]:
world_map = gpd.read_file(geodatasets.get_path('naturalearth.land'))

In [ ]:
def plot_dust_deposition_map(df, title, name_to_save, label_str='log_dep', measure_units='Dust flux log$_{10}$[g m$^{-2}$ a$^{-1}$]', limit_bar = 3.2):
    """Generate a map plot of dust deposition based on the provided dataframe.

    Parameters:
    - df (pandas DataFrame): The dataframe containing the dust deposition data.
    - title(str): The title to be displayed on the plot.
    - name_to_save (str): The filename to save the generated map plot.
    - label_str (str, optional): The column name in the dataframe to use for labeling the data on the plot. Defaults to 'log_dep'.
    - measure_units (str, optional): The units of measurement for the dust flux. Defaults to 'Dust flux log$_{10}$[g m$^{-2}$ a$^{-1}$]'.
    - limit_bar (float, optional): The limit value for the colorbar on the plot. Defaults to 3.2."""

    df_dust_geopandas = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))

    fig, ax = plt.subplots(figsize=(12, 10))

    # Set limits and ticks
    ax.set_xlim(-180, 180)
    ax.set_ylim(-90, 90)
    ax.set_xticks(np.arange(-180, 181, 45))
    ax.set_yticks(np.arange(-90, 91, 30))

    # Plot world map
    world_map.boundary.plot(ax=ax, color='black', linewidth=0.8)

    # Plot the GeoDataFrame
    df_dust_geopandas.plot(column=label_str,
                           ax=ax,
                           legend=True,
                           legend_kwds={'label': measure_units, 'orientation': "horizontal"},
                           cmap='viridis',
                           vmin=-limit_bar,
                           vmax=limit_bar,
                           linewidth=0.005,
                           markersize=100)

    # Set labels and title
    ax.set(xlabel='Longitude', ylabel='Latitude', title=title)

    # Save and show the plot
    plt.savefig(f"{name_to_save}.pdf", bbox_inches='tight', dpi=600)
    plt.show()

    return


In [ ]:
plot_dust_deposition_map(df=df_empirical_Holocene,
                         title='Holocene',
                         name_to_save='DATA_MAP_HOLOCENE')

plot_dust_deposition_map(df=df_empirical_LGM,
                         title='Last Glacial Maximum',
                         name_to_save='DATA_MAP_LGM')

## Create and visualize the global grid
The global grid for the predictions consists of 3-degree spacing.

In [ ]:
longitude_to_predict = np.linspace(-180, 180, 121)
latitude_to_predict = np.linspace(-90, 90, 61)

xx, yy = np.meshgrid(longitude_to_predict, latitude_to_predict)
XYpairs = np.vstack([ xx.reshape(-1), yy.reshape(-1) ])
df_gpd_global_grid = gpd.GeoDataFrame(geometry=gpd.points_from_xy(XYpairs[0], XYpairs[1]))
df_global_grid = pd.DataFrame({
    'lon': df_gpd_global_grid.geometry.x,
    'lat': df_gpd_global_grid.geometry.y
})

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
world_map.boundary.plot(ax=ax, color='black', linewidth=0.8)
df_gpd_global_grid.plot(ax=ax, color='red', markersize=10, marker ='o')

ax.set(xlabel='Longitude', ylabel='Latitude',
       title=f'Global grid with {df_gpd_global_grid.shape[0]} points')
plt.show()

In [ ]:
 i

def distancia_geodesica(lat1, lon1, lat2, lon2, R=6371):
    """
    Calcula la distancia geodésica entre dos puntos en una esfera
    utilizando la fórmula de la distancia geodésica.
    
    Args:
    lat1, lon1: Latitud y longitud del primer punto en grados.
    lat2, lon2: Latitud y longitud del segundo punto en grados.
    R: Radio de la esfera (opcional, por defecto 6371 km).
    
    Returns:
    La distancia geodésica entre los dos puntos en la superficie de la esfera.
    """
    # Convertir grados a radianes
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Aplicar la fórmula
    d = R * math.acos(math.sin(lat1) * math.sin(lat2) + math.cos(lat1) * math.cos(lat2) * math.cos(lon2 - lon1))
    
    return d

# Ejemplo de uso
lat1, lon1 = 40.7128, -74.0060  # Coordenadas de Nueva York
lat2, lon2 = 34.0522, -118.2437  # Coordenadas de Los Ángeles

distancia = distancia_geodesica(lat1, lon1, lat2, lon2)
print("La distancia geodésica entre Nueva York y Los Ángeles es:", round(distancia, 2), "kilómetros.")


In [ ]:
def nearest_neighbour_estimation(df, x, k, dist):
    """
    Perform a k nearest neighbour estimation.

    Parameters:
    df : pandas.DataFrame
        The database.
    x : Iterable
        Location for the estimation.
    k : int
        The number of neighbours to consider.
    dist : str
        The type of distance definition.
    """
    
    if dist == "geodesic":
        distance_fun = distance_geodesic
    else:
        distance_fun = distance_euclidean
    
    distances = np.repeat(np.inf, len(df))
    for index, row in df.iterrows():
        distances[index] = distance_fun(x, (row["lon"], row["lat"]))
    
    sorted_indices = np.argsort(distances)
    nearest_points = sorted_indices[:k]
    
    values_neighbours = df.loc[nearest_points, "log_dep"]
    estimation = np.average(values_neighbours)
    
    return estimation

In [ ]:
# Choose the number of neighbours to include in the estimation
n_neighbours = 5

In [ ]:
estimations_holocene = np.repeat(np.inf, len(df_global_grid))
estimations_lgm = np.repeat(np.inf, len(df_global_grid))

for index, row in df_global_grid.iterrows():
    point_for_estimation = (row["lon"], row["lat"])
    estimations_holocene[index] = nearest_neighbour_estimation(df_empirical_Holocene, point_for_estimation, n_neighbours, dist='euclidean')
    estimations_lgm[index] = nearest_neighbour_estimation(df_empirical_LGM, point_for_estimation, n_neighbours, dist='euclidean')

df_global_grid["est_hol"] = estimations_holocene
df_global_grid["est_lgm"] = estimations_lgm

In [ ]:
plot_dust_deposition_map(df=df_global_grid,
                         title='Holocene',
                         name_to_save='ESTIMATION_MAP_HOLOCENE',
                         label_str='est_hol')

plot_dust_deposition_map(df=df_global_grid,
                         title='Last Glacial Maximum',
                         name_to_save='ESTIMATION_MAP_LGM',
                         label_str='est_lgm')

El codigo fue sacado con ayuda de chat GTP


a) Diferencias más grandes en una parte del mundo: Las diferencias pueden ser más grandes en ciertas partes del mundo debido a la curvatura de la Tierra. En áreas cercanas a los polos, la curvatura es mayor, lo que puede hacer que las distancias geodésicas sean significativamente diferentes de las distancias euclidianas, especialmente en distancias largas.

b) Método más confiable en los polos norte y sur: La distancia geodésica es más confiable en los polos norte y sur debido a la singularidad de la longitud en el meridiano de 180 grados. En estas regiones, las distancias euclidianas pueden ser engañosas debido a la convergencia de los meridianos.

c) Método más confiable en longitud 180: En longitud 180, donde el meridiano internacional de fecha se encuentra, la distancia geodésica es más confiable debido a la singularidad de la longitud. Las distancias euclidianas pueden no reflejar con precisión la verdadera distancia debido a la discontinuidad en la representación de la longitud.

d) Diferencias entre el Holoceno y el LGM: Durante el Holoceno (época geológica actual), la diferencia entre los dos métodos puede ser menor en comparación con el Último Máximo Glacial (LGM), debido a la variación en la forma de la Tierra. Durante el LGM, la cobertura de hielo cambió la forma de la superficie terrestre, lo que puede afectar significativamente las distancias geodésicas.

e) Elección del número de vecinos cercanos: El número de vecinos cercanos depende del contexto del problema. Para algunas aplicaciones, como el análisis de datos geoespaciales, puede ser útil elegir un número suficientemente grande de vecinos para capturar la variabilidad en la distribución de los puntos. Sin embargo, elegir un número demasiado grande puede aumentar la complejidad computacional. Por lo tanto, es importante encontrar un equilibrio entre la precisión y la eficiencia computacional.
